# Introduction to Probabilistic Programming

## Introduction

Modelling with the power of Domain Driven Design => F# = Win.

## Preliminaries


### Probability

In [276]:
type Probability = private Probability of double

module Probability =
    let create ( probability : double ) : Probability option =
        if probability >= 0.0 && probability <= 1.0 then Some ( Probability probability )
        else None

    let getProbability ( Probability probability ) = probability

All branches of an 'if' expression must return values of the same type as the first branch, which here is 'Probability option'. This branch returns a value of type 'Door'.

#### Example

In [277]:
// Creating
let prob = Probability.create ( 1.0/2.0 )

// Getting the value
match prob with
    | Some p -> Probability.getProbability p
    | None   -> nan

This expression was expected to have type
    'Probability option'    
but here has type
    'Door'    

### Distribution

In [279]:
type Distribution<'T> = seq< 'T * Probability >

#### Uniform Distribution

In [278]:
let uniformRandomNumberGenerator = System.Random()

let uniformDistribution ( x : seq< int > ) : Distribution<int>  = 
    seq {
        let countOfSequence = Seq.length x
        let distributionSequence =
            x 
            |> Seq.map( fun e -> e, Probability.create ( 1.0 / double( countOfSequence )))
            |> Seq.filter( fun ( e, p ) -> p.IsSome )
            |> Seq.map( fun (e, p) -> e, p.Value )
        yield! distributionSequence
    }
    
let drawOne ( uniformDistribution : Distribution<int> ) : int * Probability = 
    let countOfDistribution = 
        Seq.length uniformDistribution
    let idx = uniformRandomNumberGenerator.Next( 0, countOfDistribution )
    uniformDistribution
    |> Seq.item idx

In [281]:
let diceRollDistribution = uniformDistribution [ 1..6 ]
printfn "%A" ( Seq.toList diceRollDistribution )

[(1, Probability 0.1666666667); (2, Probability 0.1666666667);
 (3, Probability 0.1666666667); (4, Probability 0.1666666667);
 (5, Probability 0.1666666667); (6, Probability 0.1666666667)]


In [280]:
printfn "%A" ( drawOne diceRollDistribution )

(4, Probability 0.1666666667)


## The Monty Hall Problem

The Monty Hall Problem involves a game show contestent choosing 

In [282]:
type Outcome = Win | Lose
type Door    = A | B | C | NA
let doors    = [ A; B; C ]

In [283]:
type GameState = { winningDoor : Door; chosenDoor : Door; openedDoor : Door }

let start : GameState  = 
    { winningDoor = NA; chosenDoor = NA; openedDoor = NA }

In [284]:
let hidePrize ( state : GameState ) : GameState =
    let winningDoorIdx = fst ( drawOne ( uniformDistribution [ 1..3 ] )) - 1
    { state with winningDoor = doors.[ winningDoorIdx ] }
    
hidePrize start

{winningDoor = C;
 chosenDoor = NA;
 openedDoor = NA;}

In [285]:
let initializeGame : GameState =
    hidePrize start
    
initializeGame

{winningDoor = B;
 chosenDoor = NA;
 openedDoor = NA;}

In [286]:
let chooseDoor ( state : GameState ) ( door : Door ) : GameState =
    { state with chosenDoor = door }
    
let chooseRandomDoor ( state : GameState ) : GameState =
    let chosenDoorIdx = fst ( drawOne ( uniformDistribution [ 1..3 ] )) - 1
    { state with chosenDoor = doors.[ chosenDoorIdx ] }

let chosenRandomDoor = chooseRandomDoor initializeGame
chosenRandomDoor

{winningDoor = B;
 chosenDoor = A;
 openedDoor = NA;}

In [287]:
let openDoor ( state : GameState ) : GameState =
    // Choose the Non-Winning Door that hasn't been chosen by the contestant.
    let doorToOpenAsList = 
        doors 
        |> List.except [ state.winningDoor; state.chosenDoor ]
    let doorToOpen = doorToOpenAsList.[0]
    { state with openedDoor = doorToOpen }
    
let postOpenDoor = openDoor ( chosenRandomDoor )
postOpenDoor

{winningDoor = B;
 chosenDoor = A;
 openedDoor = C;}

In [288]:
type Strategy = GameState -> Outcome

let switch ( state : GameState ) : Outcome =
    let doorToSwitchTo = 
        doors
        |> List.except [ state.chosenDoor; state.openedDoor ]
    if doorToSwitchTo.[0] = state.winningDoor then Win
    else Lose
    
let stay ( state : GameState ) : Outcome = 
    if state.chosenDoor = state.winningDoor then Win
    else Lose

printfn "On Switch: %A" ( switch postOpenDoor )
printfn "On Stay: %A"   ( stay postOpenDoor )

On Switch: Win
On Stay: Lose


In [289]:
let simulateMontyHall ( strategy : Strategy ) : Outcome = 
    let game = 
        initializeGame 
        |> chooseRandomDoor
        |> openDoor
    strategy( game )

simulateMontyHall switch

Win

In [308]:
let generateDistributionOfStaying ( numberOfTrials : int ) : Outcome list = 
    let mutable list = []
    for i in 1 .. numberOfTrials do
        list <- list @ [ simulateMontyHall stay ] 
    list

let distributionOfWinning =
    generateDistributionOfStaying 10000

In [327]:
#load "Paket.fsx"

Paket.Package(["FsLab"])

This expression was expected to have type
    'string option'    
but here has type
    'Door'    
This expression was expected to have type
    'string option'    
but here has type
    'Door'    
This expression was expected to have type
    'string option'    
but here has type
    'Door'    
This expression was expected to have type
    'string option'    
but here has type
    'Door'    

In [307]:
let generateDistributionOfSwitching ( numberOfTrials : int ) : Outcome list = 
    let mutable list = []
    for i in 1 .. numberOfTrials do
        list <- list @ [ simulateMontyHall switch ] 
    list
    
let distributionOfSwitching = 
    generateDistributionOfSwitching 10000

## Bayesian Inference

### A/B Test

In [111]:
let posteriorDistribution ( data : double                ) 
                          ( priorSampler : seq<double>   ) 
                          ( simulator : double -> double ) : seq<double option> = 
    seq {
        for p in priorSampler do
            printfn "Simulator: %A" ( simulator p )
            printfn "p: %A" ( p )
            if simulator( p ) = data then yield Some p
            else yield None
    }

This expression was expected to have type
    'double option'    
but here has type
    'Door'    

In [61]:
open System

let seed   = 123 
let random = Random( seed )

let simulateConversion ( p : double ) ( nVisitors : int ) : double = 
    [ 1..nVisitors ]
    |> List.map( fun x -> random.NextDouble() )
    |> List.filter( fun d -> d < p )
    |> List.sum

// Print out the Simulated Conversions.
printfn "%f" ( simulateConversion 0.1 1000 )
printfn "%f" ( simulateConversion 0.1 1000 )
printfn "%f" ( simulateConversion 0.1 1000 )

5.380237
5.741275
4.741965


In [143]:

// [ 0, 1 ]
let uniformPriorSampler =
    seq { while true do yield random.NextDouble() } 
    
Seq.take 3 uniformPriorSampler

The value, namespace, type or module 'random' is not defined. Maybe you want one of the following:
   Random

In [135]:
let applySimulation ( p : double ) : double  =
    simulateConversion p 100
    
let posterior = 
    posteriorDistribution 0.4 uniformPriorSampler applySimulation
let r = 
    posterior
    |> Seq.take 1000
    |> Seq.filter( fun x -> x.IsSome )
    |> Seq.toList

printfn "%A" r

Simulator: 2.846242449
p: 0.2627974554
Simulator: 36.90148848
p: 0.874018535
Simulator: 0.7301468564
p: 0.1267017597
Simulator: 8.801800571
p: 0.4578643485
Simulator: 0.3914985361
p: 0.1204433409
Simulator: 1.190109453
p: 0.1702598418
Simulator: 23.25329641
p: 0.6804495424
Simulator: 26.08825332
p: 0.6744963027
Simulator: 22.36363924
p: 0.6454630562
Simulator: 0.5832528419
p: 0.09421572652
Simulator: 15.81285554
p: 0.5954175482
Simulator: 3.03254701
p: 0.2701967588
Simulator: 30.01608835
p: 0.837746879
Simulator: 0.9326540269
p: 0.1163983555
Simulator: 0.2873933177
p: 0.07534070363
Simulator: 29.26479057
p: 0.762166235
Simulator: 9.279476567
p: 0.4401206819
Simulator: 10.07707203
p: 0.4504454189
Simulator: 0.0008878833618
p: 0.01909649606
Simulator: 48.74211227
p: 0.9243995761
Simulator: 12.44461781
p: 0.5335058703
Simulator: 49.40923668
p: 0.9702426041
Simulator: 0.1466603857
p: 0.04529752305
Simulator: 45.00067978
p: 0.9343386846
Simulator: 1.935621629
p: 0.2129371889
Simulator: 0.18